In [8]:
%cd  G:\Shared drives\INVent\mfk58\Dataset\

%pip install numpy
%pip install SimpleITK

G:\Shared drives\INVent\mfk58\Dataset
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\theev\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\theev\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [9]:
import glob
import SimpleITK as sitk
import numpy as np
def hasMask(name):
    files = glob.glob(name+"/*.mha")

    reader = sitk.ImageFileReader()

    if(len(files)<2):
        return False

    reader.SetFileName(files[0])
    reader.LoadPrivateTagsOn()
    reader.ReadImageInformation()

    size1 = reader.GetSize()  
    res1 = reader.GetSpacing() 

    
    reader.SetFileName(files[1])
    reader.LoadPrivateTagsOn()
    reader.ReadImageInformation()

    size2 = reader.GetSize()  
    res2 = reader.GetSpacing()
    
    if not (size1 == size2 and res2 == res2):
        return False


    return True

def axial(name):
    files = glob.glob(name+"/*.mha")
    if("ax" in files[0] and "ax" in files[1]):
        return True
    return False



CCF_count = 0

CCF_missing = [] #missing masks
CCF_list = []

for name in glob.glob('G:/Shared drives/INVent/mfk58/Dataset/CCF_Pre_Resampled/*'):
    
    if not (hasMask(name)):
        CCF_missing.append(name)
        continue
    if axial(name):
        CCF_count +=1
        CCF_list.append(name)
    

UH_count = 0

UH_missing = [] #missing masks
UH_list = []

for name in glob.glob('G:/Shared drives/INVent/mfk58/Dataset/UH_Pre_Resampled/*'):
    

    if not (hasMask(name)):
        UH_missing.append(name)
        continue
    if axial(name):
        UH_count +=1
        UH_list.append(name)

comList = CCF_list+UH_list

KeyboardInterrupt: 

In [ ]:
def get_bounding_box(x):
    """ Calculates the bounding box of a ndarray"""
    mask = x == 0
    bbox = []
    all_axis = np.arange(x.ndim)
    for kdim in all_axis:
        nk_dim = np.delete(all_axis, kdim)
        mask_i = mask.all(axis=tuple(nk_dim))
        dmask_i = np.diff(mask_i)
        idx_i = np.nonzero(dmask_i)[0]
        
        bbox.append(slice(min(idx_i)+1, max(idx_i)+1))
    return bbox


def filterA(a,val):
    arr = a
    arr[arr != val] = 0
    return arr

In [ ]:
assert len(comList) == 166


excl = []

FOV5p = []
FOV8p = []
FOV10p = []
FOV12p = []
FOV15p  =[]
FOV17p =[]
FOV20p = []
allElse = []

for name in comList:
    #print("running " +name)
    files = glob.glob(name+"/*.mha")
    if not ("ax" in files[0] and "ax" in files[1]):
        continue
    for file in files:
        if not ("label" in file):
            continue

        if file in excl:
            continue

        image = sitk.ReadImage(file, imageIO="MetaImageIO")
        arr = sitk.GetArrayFromImage(image)

        img_size = arr.shape

        #arr = filterA(arr,1)
        
        bbx = get_bounding_box(arr)
        
        arr = arr[bbx[0].start:bbx[0].stop,bbx[1].start:bbx[1].stop,bbx[2].start:bbx[2].stop]
        size = arr.shape
        
        if (size[1]*size[2]) < (0.05*(img_size[1]*img_size[2])):
            FOV5p.append(name)
        elif (size[1]*size[2]) < (0.08*(img_size[1]*img_size[2])):
            FOV8p.append(name)
        elif (size[1]*size[2]) < (0.10*(img_size[1]*img_size[2])):
            FOV10p.append(name)
        elif (size[1]*size[2]) < (0.12*(img_size[1]*img_size[2])):
            FOV12p.append(name)
        elif (size[1]*size[2]) < (0.15*(img_size[1]*img_size[2])):
            FOV15p.append(name)
        elif (size[1]*size[2]) < (0.17*(img_size[1]*img_size[2])):
            FOV17p.append(name)
        elif (size[1]*size[2]) < (0.20*(img_size[1]*img_size[2])):
            FOV20p.append(name)
        else:
            allElse.append(name)
        


        

In [ ]:
print(len(FOV5p))
print(len(FOV5p+FOV8p))
print(len(FOV5p+FOV8p+FOV10p))
print(len(FOV5p+FOV8p+FOV10p+FOV12p))
print(len(FOV5p+FOV8p+FOV10p+FOV12p+FOV15p))
print(len(FOV5p+FOV8p+FOV10p+FOV12p+FOV15p+FOV17p))
print(len(FOV5p+FOV8p+FOV10p+FOV12p+FOV15p+FOV17p+FOV20p))
largeFOV = []

11
20
36
57
88
105
122


In [ ]:
import SimpleITK as sitk
import os


#Try to use absolute paths as much as possible

#img: string with filepath to main image
#mask: string with filepath to segmentation
#labeldir: string with filepath to labels
#tempdir: string with filepath that you don't mind having a random file be written to, defaults to just C: - good idea to change it
    #labeldir and tempdir must be in the main C: drive - workarounds were made to use img and mask with G: drive for convenience
def display(img, mask,labeldir,tempdir="C:\\",name="temp viewer"):
    v = sitk.ImageViewer()
    r = sitk.ImageFileReader()
    r.SetFileName(img)
    image = r.Execute()

    r.SetFileName(mask)
    w = sitk.ImageFileWriter()
    
    os.chdir(tempdir)
    w.SetFileName("temp.mha")
    w.Execute(r.Execute())

    v.SetApplication(r'C:\Program Files\ITK-SNAP 3.8\bin\ITK-SNAP')
    s = ""
    if labeldir != None:
        s = ' -l '+ labeldir
    v.SetCommand('itk-snap -s '+ tempdir + '\\temp.mha' + s + " -g") #this requires the ITK-SNAP directory to be added to the PATH env var.
    v.SetTitle(name)
    v.Execute(image)
    


In [ ]:
list = [FOV5p,FOV8p,FOV10p,FOV12p,FOV15p,FOV17p,FOV20p,allElse]




for l in list:
    lf = []
    for dir in l:
        files = glob.glob(dir+"/*.mha")
        label = ""
        img = ""
        if "label" in files[0]:
            label = files[0]
            img = files[1]
        else:
            label = files[1]
            img = files[0]

        display(img, label, "C:\\Users\\theev\\Downloads\\Rectal_Data_Labels.label", "C:\\Users\\theev\\Documents\\LAB",img.split("\\")[-1])

        print("")
        ans = input("Large? y/n")
        if ans == "y":
            lf.append(dir)
            print("Added" + img)

    largeFOV.append(lf)




AddedG:/Shared drives/INVent/mfk58/Dataset/CCF_Pre_Resampled\RectalCA_029\CCF_RectalCA_029_pre_ax_raw_resampled.mha

AddedG:/Shared drives/INVent/mfk58/Dataset/CCF_Pre_Resampled\RectalCA_028\CCF_RectalCA_028_pre_ax_raw_resampled.mha

AddedG:/Shared drives/INVent/mfk58/Dataset/UH_Pre_Resampled\RectalCA_015\UH_RectalCA_015_pre_ax_raw_resampled.mha






AddedG:/Shared drives/INVent/mfk58/Dataset/UH_Pre_Resampled\RectalCA_112\UH_RectalCA_112_pre_ax_raw_resampled.mha



AddedG:/Shared drives/INVent/mfk58/Dataset/CCF_Pre_Resampled\RectalCA_018\CCF_RectalCA_018_pre_ax_raw_resampled.mha

AddedG:/Shared drives/INVent/mfk58/Dataset/CCF_Pre_Resampled\RectalCA_040\CCF_RectalCA_040_pre_ax_raw_resampled.mha

AddedG:/Shared drives/INVent/mfk58/Dataset/CCF_Pre_Resampled\RectalCA_060\CCF_RectalCA_060_pre_ax_resampled.mha

AddedG:/Shared drives/INVent/mfk58/Dataset/CCF_Pre_Resampled\RectalCA_110\CCF_RectalCA_110_pre_ax_raw_resampled.mha

AddedG:/Shared drives/INVent/mfk58/Dataset/UH_Pre_Resampled\Rect

In [12]:
#ones I accidentially mislabeled because of user input error
remove = [r"G:/Shared drives/INVent/mfk58/Dataset/CCF_Pre_Resampled\RectalCA_124",r"G:/Shared drives/INVent/mfk58/Dataset/UH_Pre_Resampled\RectalCA_173",r"G:/Shared drives/INVent/mfk58/Dataset/UH_Pre_Resampled\RectalCA_018"]

for r in remove:
    for list in largeFOV:
        if not r in list:
            continue
        list.remove(r)

In [20]:
%pip install pandas
%pip install xlsxwriter
%cd G:\Shared drives\INVent\mfk58

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\theev\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.
G:\Shared drives\INVent\mfk58


You should consider upgrading via the 'c:\Users\theev\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [26]:
import pandas as pd

list = [FOV5p,FOV8p,FOV10p,FOV12p,FOV15p,FOV17p,FOV20p,allElse]
df = pd.DataFrame(largeFOV).T
writer = pd.ExcelWriter('large-FOVs.xlsx', engine='xlsxwriter')
df.to_excel(writer, sheet_name='raw list', index=False)
df = pd.DataFrame(list).T
df.to_excel(writer, sheet_name='percent size filter', index=False)
writer.close()